the purpose of this notebook is to take the big dataframe created in 07.01-baseline_data_merging, and make an aggregate we can use for our baseline "worst case" scenario model. This will then be fed into R, where we will use the MICE package to impute data

changelog:

* 4/16/19: added newagg
* 4/17/19: reformatted the ordering of how code runs, and variable names. added aggregation #3.
* 4/19/19: changed the standardizing so that log(x+1) is now applied prior to standardization. also removed ordinal variables from standardizing algorithm and concat them in later with median 0 and iqr 1 so standardize value is either 0 or 1. values from ordinal are not log transformed.


## last run: 6/9/19: sensitivity analysis 1day timewindow

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/04042019_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals, 14478 icustay_id
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 92.1 ms


In [3]:
#importing in all clinical_variable files
timewindowdays="_24"
# date= '04042019'
date='09062019'
os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/')
big_df= pd.read_csv(Path(wd+'/data/processed/merged/{}_longdf_preImp{}.csv'.format(date,timewindowdays),  index_col=0))
big_df= big_df.reset_index(drop=True).iloc[:,1:]


time: 5.15 s


In [4]:
## last minute data cleaning/formatting

# #removing firstpos else neg ssc col
# big_df=big_df.loc[:,list(big_df.columns!="first_pos_else_neg_ssc")]
# #changing pao2/fio2 ratio label
# big_df.loc[big_df['label']=="pao2/fio2",'label']="pao2fio2Ratio"

time: 882 µs


In [5]:
#overview of all variables and formats
big_df.groupby('label')['value'].describe().sort_values('unique')

,count,unique,top,freq
label,,,,
leukocyte,14181,2,Neg/Not_tested,12594
norepinephrine,23243,2,0.0,11710
vasopressin,15180,2,0.0,13889
o2_flow,14277,2,0.0,13766
cancer_elix,14181,2,0.0,12439
gender,14181,2,M,7776
rrt,14181,2,0.0,13235
phenylephrine,19830,2,0.0,12317
nitrite,14181,2,Neg/Not_tested,13858


time: 792 ms


## initial data prep

In [6]:
# #converting venttype to category
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')
# #big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].apply(astype('category'))
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'] = pd.Categorical(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])

time: 790 µs


In [7]:
#converting categories to integers
mapper={'Mech':2 , 'Oxygen': 1, 'None': 0}
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']=big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].replace(mapper)

#gender_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='gender','value'])[1]
mapper={'F':0 , 'M': 1}
big_df.loc[big_df.loc[:,'label']=='gender','value']=big_df.loc[big_df.loc[:,'label']=='gender','value'].replace(mapper)

#leukocyte_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[1]
mapper={'Neg/Not_tested':0 , 'pos': 1}
big_df.loc[big_df.loc[:,'label']=='leukocyte','value']=big_df.loc[big_df.loc[:,'label']=='leukocyte','value'].replace(mapper)
#     pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[0] 

#nitrite_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='nitrite','value'])[1]
mapper={'Neg/Not_tested':0 , 'pos': 1}
big_df.loc[big_df.loc[:,'label']=='nitrite','value']=big_df.loc[big_df.loc[:,'label']=='nitrite','value'].replace(mapper)
#     pd.factorize(big_df.loc[big_df.loc[:,'label']=='nitrite','value'])[0] 


time: 1.65 s


In [8]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].value_counts()

2    6772
1    6417
0     802
Name: value, dtype: int64

time: 132 ms


In [9]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].value_counts()

2    6772
1    6417
0     802
Name: value, dtype: int64

time: 129 ms


#exploring alternative 

# changing pao2/fio2 ratio to a category
A PaO2/FiO2 ratio less than or equal to 200 is necessary for the diagnosis of acute respiratory distress syndrome by the AECC criteria.[6] 
The more recent Berlin criteria defines mild ARDS at a ratio of <300.

A PaO2/FiO2 ratio less than or equal to 250 is one of the minor criteria for severe community acquired pneumonia (i.e., possible indication for inpatient treatment).

A PaO2/FiO2 ratio less than or equal to 333 is one of the variables in the SMART-COP risk score for intensive respiratory or vasopressor support in community-acquired pneumonia.



In [10]:
pd.to_numeric(big_df[big_df['label']=="pao2fio2ratio"]['value'])
labels= ['0-200', '201-333', "334-475","476+"]
bins = pd.IntervalIndex.from_tuples([(0, 200), (200, 333), (333, 475),(475,3000)])

big_df.loc[big_df['label']=="pao2fio2ratio",'value']=pd.cut(pd.to_numeric(big_df.loc[big_df['label']=="pao2fio2ratio",'value']), bins,right=False, labels=labels)
big_df.loc[big_df['label']=="pao2fio2ratio",'value'].value_counts()


(475, 3000]    10378
(0, 200]        1364
(333, 475]      1267
(200, 333]      1172
Name: value, dtype: int64

time: 818 ms


# investigating the dataframe time window between t_0 and t_0 + w, 
* where w is the window length. default was 72, with 48 and 24 being used in the sensitivity analysis

In [11]:
big_df.head()

,icustay_id,delta,value,uom,t_0,label,source,subject_id
0,200012.0,0 days 00:00:00.000000000,0.0,y/n,2153-12-23,cancer_elix,cancer_elix,28448
1,200012.0,0 days 00:00:00.000000000,1,daily_sofa_score,2153-12-23,daily_sofa,sofa,28448
2,200012.0,0 days 00:00:00.000000000,0.0,y/n,2153-12-23,dobutamine,dobutamine,28448
3,200012.0,0 days 00:00:00.000000000,0.0,y/n,2153-12-23,dopamine,dopamine,28448
4,200012.0,0 days 00:00:00.000000000,0.0,y/n,2153-12-23,epinephrine,epinephrine,28448


time: 9.77 ms


In [13]:
pd.to_timedelta(big_df['delta']).describe()

count                   3204122
mean     0 days 11:05:57.807424
std      0 days 07:25:15.262627
min             0 days 00:00:00
25%             0 days 04:20:00
50%             0 days 11:00:00
75%             0 days 17:00:00
max             2 days 00:00:00
Name: delta, dtype: object

time: 10.5 s


In [ ]:
(pd.to_datetime(final_pt_df2['t_0_sc'])-pd.to_datetime(final_pt_df2['ICU_admit'])).describe() #icu admit time - ssc time

In [14]:
len(final_pt_df2)

14181

time: 1.81 ms
